In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=38c7c58d632a07825f810d5dd4e914867bbacf3a4916d4ea5092f004f7881162
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark



**INCREMNTAL LOAD**

In [8]:
# initialize spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sparksql_Example').getOrCreate()

# Full refresh : Load the entire Database
df_sales = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/content/sample_data/sample_data.txt")

# Apply transformations (If necessary)
df_transformed = df_sales.withColumn("Total_Sales",df_sales['quantity'] * df_sales['price'])

# Incremental Load : partition the data by "Date" and appending new data
output_path = "/content/sample_data/people.txt"
df_transformed.write.partitionBy("date").mode("append").parquet(output_path)

# verify partitioned data
partitioned_df = spark.read.parquet(output_path)
partitioned_df.show()

+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|transaction_id|customer_id| product|quantity|price|         updated_at|Total_Sales|      date|
+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|             1|        101|  Laptop|       1| 1000|2024-09-01 08:00:00|       1000|2024-09-01|
|             2|        102|   Phone|       2|  500|2024-09-01 09:00:00|       1000|2024-09-01|
|             1|        101|  Laptop|       1| 1000|2024-09-01 08:00:00|       1000|2024-09-01|
|             2|        102|   Phone|       2|  500|2024-09-01 09:00:00|       1000|2024-09-01|
|             5|        105|Keyboard|       1|   50|2024-09-03 12:00:00|         50|2024-09-03|
|             6|        106|   Mouse|       3|   30|2024-09-03 13:00:00|         90|2024-09-03|
|             5|        105|Keyboard|       1|   50|2024-09-03 12:00:00|         50|2024-09-03|
|             6|        106|   Mouse|   